In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2018)

In [27]:
trn = pd.read_csv("../dataset/train_ver2.csv")
tst = pd.read_csv("../dataset/test_ver2.csv")

/var/folders/yx/5tk14qzj4n59c7c0rpmfb5qr0000gn/T/ipykernel_20677/2781771228.py:1: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  trn = pd.read_csv("../dataset/train_ver2.csv")
/var/folders/yx/5tk14qzj4n59c7c0rpmfb5qr0000gn/T/ipykernel_20677/2781771228.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  tst = pd.read_csv("../dataset/test_ver2.csv")


In [28]:
## データの前処理

prods = trn.columns[24:].tolist()
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

features = []

categorical_cols = ["ind_empleado", "pais_residencia", "sexo", "tiprel_1mes", "indresi", "indext", "conyuemp", "canal_entrada", "indfall", "tipodom", "nomprov", "segmento"]
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

In [29]:
df["age"] = df["age"].str.replace(" ", "")
df["age"] = df["age"].replace("NA", -99)
df["age"] = df["age"].fillna(-99)
df["age"] = df["age"].astype(np.int8)
display(df["age"].unique())

array([ 35,  23,  22,  24,  65,  28,  25,  26,  53,  27,  32,  37,  31,
        39,  63,  33,  55,  42,  58,  38,  50,  30,  45,  44,  36,  29,
        60,  57,  67,  47, -99,  34,  48,  46,  54,  84,  15,  12,   8,
         6,  83,  40,  77,  69,  52,  59,  43,  10,   9,  49,  41,  51,
        78,  16,  11,  73,  62,  66,  17,  68,  82,  95,  96,  56,  61,
        79,  14,  19,  13,  86,  64,  20,  72,  89,  71,   7,  70,  74,
        21,  18,  75,   4,  80,  81,   5,  76,  92,  93,  85,  91,  87,
        90,  94,  99,  98,  88,  97, 100, 101, 106, 103,   3,   2, 102,
       104, 111, 107, 109, 105, 112, 115, 110, 116, 108, 113], dtype=int8)

In [30]:
df["antiguedad"] = df["antiguedad"].str.replace(" ", "")
df["antiguedad"] = df["antiguedad"].replace("NA", -99)
df["antiguedad"] = df["antiguedad"].fillna(-99)
df["antiguedad"] = df["antiguedad"].astype(np.int8)
display(df["antiguedad"].unique())

array([   6,   35,   34,  -99,   33,   31,   21,   16,   27,    9,   22,
         13,   29,    8,   11,   10,   28,   24,    7,   25,   14,   12,
         26,   23,    1,   18,    4,    3,   17,   32,   20,   15,   30,
         19,   36,   40,   38,   37,   39,    0,    5,   47,   44,   42,
         46,   45,   43,   41,   57,   48,   52,   49,   50,   56,   58,
         51,   55,   54,   53,   59,   62,   61,   60,   63,    2, -117,
        -91,  118,  -92,   94,  -97, -113,  105, -105,  -94, -119, -106,
       -128,  122, -100,  119,  -96,   79,   95, -124,  -95,   98,  127,
         72, -101,  108,  -93,  102, -108,  115, -110,  107,   81,  -40,
       -121,   92,  121,  -58, -122,   93, -116,  110,  120, -109,   64,
         77,   85,   99,   78,  100,  113, -102,  -90,  124, -115,   66,
        117,   86,  -63,   80, -112,   87,  126,  -98,  101,  116,  -21,
         88, -111,  103, -107,  109, -125,   97, -123,   68,   84,  -24,
        125,  -79,  112,   96,   69,  -85, -114,  -

In [35]:
df["renta"] = df["renta"].str.replace(" ", "")
df["renta"] = df["renta"].replace("NA", -99)
df["renta"] = df["renta"].fillna(-99)
df["renta"] = df["renta"].astype(float).astype(np.int8)
display(df["renta"].unique())

array([ -99,  -20,  -78, -123, -117,  -30,  -97,  109,  -74,  -62,   62,
         45,   78,  -94,    2,  -72,  -38,  -12,  -43,   30, -121,  117,
       -122,  120,   44,  110,  -89,   64,   76,  -18,  -73,  116, -128,
       -113,  -95,   83,   -1,   31,  -37,  100,  104,  -46,  -55,   25,
        -44, -106,  -84,  -11,  108,   90,  -87,   61,  -49,   96,   39,
         26,  -90,   73, -110,   -7,   15,  -24, -105,   37,  107,   -4,
         69,   81,   19,   97,  -29,   72,   41,   75,  -40,    1,   77,
        -79,   14,  127,  -67,  106,   53,   87,  126,  103,  -86,   35,
         94, -108,    7,   55,   65,  101,   99,  -36,   12, -102,  102,
         50, -114,  -76,  -85, -116,  113,  115,   92,   43,   -3,   68,
        -88, -119,   84,   28,   17,   52, -126,   -9,   82,   23,  111,
          4,   11,  -51,   88,  -68, -109,  114,  -39,   13,  -48,  -16,
        118,   91,   86,   33,   48,   67,  -14,   66,   49,  122,   -6,
         20,   -5,  -56,  -25,   98,  -64,  -47,  -

In [41]:
df = df.copy()
df["indrel_1mes"] = df["indrel_1mes"].replace("P", 5)
df["indrel_1mes"] = df["indrel_1mes"].fillna(-99)
df["indrel_1mes"] = df["indrel_1mes"].astype(float).astype(np.int8)
display(df["indrel_1mes"].unique())
features += ["age", "antiguedad", "renta", "ind_nuevo", "indrel", "indrel_1mes", "ind_actividad_cliente"]

array([  1, -99,   3,   2,   5,   4], dtype=int8)

In [42]:
df["fecha_alta_month"] = df["fecha_alta"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[1])).astype(np.int8)
df["fecha_alta_year"] = df["fecha_alta"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[0])).astype(np.int16)
features += ["fecha_alta_month", "fecha_alta_year"]

In [43]:
df["ult_fec_cli_1t_month"] = df["ult_fec_cli_1t"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[1])).astype(np.int8)
df["ult_fec_cli_1t_year"] = df["ult_fec_cli_1t"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[0])).astype(np.int16)
features += ["ult_fec_cli_1t_month", "ult_fec_cli_1t_year"]

In [44]:
df.fillna(-99, inplace=True)

def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")]
    int_date = (int(Y) - 2015) * 12 * int(M)
    return int_date

trn["int_date"] = trn["fecha_dato"].map(date_to_int).astype(np.int8)

df_lag = df.copy()
df_lag.columns = [col + "_prev" if col not in ["ncodpers", "int_date"] else col for col in df.columns]
df_lag["int_date"] += 1

df_trn = df.merge(df_lag, on=["ncodepers", "int_date"], how="left")

del df, df_lag

for prod in prods:
    prev = prod + "_prev"
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

features += [feature + "_prev" for feature in features]
features += [prod + "_prev" for prod in prods]

KeyError: 'int_date'

In [ ]:
## モデル学習

use_dates = ["2016-01-28", "2016-02-28", "2016-03-28", "2016-04-28"]
trn = df_trn[df_trn["fecha_dato"].isin(use_dates)]
tst = df_trn[df_trn["fecha_dato"] == "2016-05-28"]

del df_trn

X = []
Y = []
for i, prod in enumarate(prods):
    prev = prod + "_prev"
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY["y"] = Y

vld_date = XY[XY["fecha_dato"] != vld_date]
vld_vld = XY[XY["fecha_dato"] == vld_date]

param = {
    "booster": "gbtree",
    "max_depth": 8,
    "nthread": 4,
    "num_class": len(prods),
    "objective": "multi:softprob",
    "silent": 1,
    "eval_metric": "mlogloss",
    "eta": 0.1,
    "min_child_weight": 10,
    "colsample_bytree": 0.8,
    "colsample_bylevel": 0.9,
    "seed": 2018,
}

X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=["y"])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=["y"])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

watch_list = [(dtrn, "train"), (dvld, "eval")]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

import pickle
pickle.dump(model, open("model/xgb.baseline.pkl", "wb"))
best_ntree_limit = model.best_ntree_limit